# 🌸 #30DayMapChallenge 2024 - Day 4: H3 Hexagons

## 桜マッピング: Mapping Tokyo's Cherry Blossoms with H3 🗼

### 1. Understanding H3: Uber's Hexagonal Hierarchical Spatial Index 🔷

#### What is H3?
* Developed by Uber in 2018
* Hierarchical spatial indexing system based on hexagons
* Open-sourced since launch
* Designed for geospatial analysis and visualization
* Now widely used in industry for location intelligence

#### Why Hexagons?
* Most efficient shape for dividing a surface
* All neighbors are equidistant
* Minimizes edge distortions
* Better distance representation than square grids
* Optimal for density analysis and clustering

#### H3 Resolution Hierarchy
* 16 resolution levels (0-15)
* Resolution 0: Hexagons ~4M km²
* Resolution 15: Hexagons ~1m²
* Each level subdivides hexagons into 7 smaller ones
* Most common resolutions for urban analysis: 8-9

### 2. Original Use Case: Uber's Implementation 🚗

#### Problems It Solved
* Dynamic pricing optimization
* Demand forecasting
* Driver allocation
* Route planning
* Real-time market analysis

#### Benefits for Uber
* Global consistency
* Computational efficiency
* Spatial analysis precision
* System scalability
* Cross-platform compatibility

### 3. Today's Challenge: Mapping Tokyo's Sakura Density 🌸

#### Data Sources
* Google Places API for tree locations
* OpenStreetMap for base data
* Historical blooming data
* Local government tree surveys

### 4. Code Implementation 💻

In [1]:
# Required package installations
!pip install h3 googlemaps folium shapely -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 9.8 MB/s eta 0:00:00


In [2]:
import googlemaps
import pandas as pd
import h3
import folium
from shapely.geometry import Point, Polygon
import numpy as np
import os
from datetime import datetime
import time

In [3]:
def get_sakura_locations(api_key, location='Tokyo, Japan'):
    """
    Fetches cherry blossom viewing locations using Google Places API.

    This function searches for various cherry blossom and sakura-related locations
    in a specified area using multiple keywords in English, Japanese, and Spanish.

    Parameters:
    -----------
    api_key : str
        Google Maps API key for authentication
    location : str, optional
        Target location to search around (default: 'Tokyo, Japan')

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing location details with columns:
        - name: Place name
        - lat: Latitude
        - lng: Longitude
        - rating: Google Maps rating
        - user_ratings_total: Total number of user ratings

    Notes:
    ------
    - Uses a 90km radius for the initial search
    - Implements pagination to get all available results
    - Includes multilingual keywords for comprehensive coverage
    - Preserves Japanese terms (桜, 花見, etc.) for cultural accuracy
    """
    # Initialize Google Maps client
    gmaps = googlemaps.Client(key=api_key)

    # Get geographical coordinates for the specified location
    geocode_result = gmaps.geocode(location)
    if not geocode_result:
        raise ValueError(f"Could not find location: {location}")

    lat = geocode_result[0]['geometry']['location']['lat']
    lng = geocode_result[0]['geometry']['location']['lng']

    # Define search keywords in multiple languages
    keywords = [
        'cherry blossom', 'sakura', '桜', 'cherry tree', 'cherry flowers',
        'blossom viewing', 'hanami spots', '花見', '桜の名所',
        'サクラスポット', 'flor de cerezo', 'árbol de sakura',
        'árbol de cerezo japonés'
    ]
    locations = []

    # Iterate through each keyword and collect locations
    for keyword in keywords:
        result = gmaps.places_nearby(
            location=(lat, lng),
            radius=90000,  # 90km radius
            keyword=keyword
        )

        # Handle pagination for complete results
        while True:
            for place in result.get('results', []):
                loc = place['geometry']['location']
                locations.append({
                    'name': place['name'],
                    'lat': loc['lat'],
                    'lng': loc['lng'],
                    'rating': place.get('rating', 0),
                    'user_ratings_total': place.get('user_ratings_total', 0)
                })

            # Check for additional pages of results
            if 'next_page_token' in result:
                time.sleep(2)  # Required delay between pagination requests
                result = gmaps.places_nearby(
                    location=(lat, lng),
                    radius=20000,
                    keyword=keyword,
                    page_token=result['next_page_token']
                )
            else:
                break

    return pd.DataFrame(locations)

In [4]:
def create_density_map(df, resolution=9):
    """
    Creates a H3 hexagon density map from location data points.

    This function converts geographical points into H3 hexagonal cells and
    counts the number of points within each hexagon to create a density map.

    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing location data with 'lat' and 'lng' columns
    resolution : int, optional
        H3 grid resolution (0-15), where higher numbers create smaller hexagons
        Default is 9, which creates hexagons ~0.1 km² in size

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing:
        - h3_index: Unique identifier for each H3 hexagon
        - count: Number of points within each hexagon

    Notes:
    ------
    - H3 resolution ranges from 0 (largest) to 15 (smallest)
    - Resolution 9 is typically good for city-level analysis
    """
    # Convert geographic coordinates to H3 indices using the new H3 syntax
    df['h3_index'] = df.apply(
        lambda row: h3.latlng_to_cell(row['lat'], row['lng'], resolution),
        axis=1
    )

    # Count the number of points falling within each hexagon
    hex_counts = df['h3_index'].value_counts()

    # Create a DataFrame with hexagon indices and their point counts
    hex_df = pd.DataFrame({
        'h3_index': hex_counts.index,
        'count': hex_counts.values
    })

    return hex_df

In [5]:
def visualize_density_with_title(hex_df, center_lat, center_lng, japanese_title="🎌 桜の名所 🌸", english_title="🌳 Cherry Blossom Viewing Spots 🍒"):
    """
    Creates an interactive Folium map showing density of cherry blossom viewing locations with bilingual titles.

    Parameters:
    -----------
    hex_df : pandas.DataFrame
        DataFrame containing H3 hexagon data with columns:
        - h3_index: H3 cell identifier
        - count: Number of points in each hexagon
    center_lat : float
        Latitude for map center
    center_lng : float
        Longitude for map center
    japanese_title : str, optional
        Title in Japanese (default: "🎌 桜の名所 🌸")
    english_title : str, optional
        Title in English (default: "🌳 Cherry Blossom Viewing Spots 🍒")

    Returns:
    --------
    folium.Map
        Interactive map with hexagonal density visualization

    Notes:
    ------
    - Uses a pink color gradient to represent density
    - Lighter pink indicates lower density, darker pink indicates higher density
    - Includes popup information showing count for each hexagon
    """
    # Initialize the base map with light theme
    m = folium.Map(
        location=[center_lat, center_lng],
        zoom_start=12,
        tiles='cartodbpositron'
    )

    # Define color scale function for density visualization
    max_count = hex_df['count'].max()
    def get_color(count):
        # Create pink gradient where higher density = darker pink
        base = int(255 - (count / max_count * 155))
        return f'#{255:02x}{base:02x}{base:02x}'

    # Add density hexagons to the map
    for _, row in hex_df.iterrows():
        # Convert H3 index to geographical boundaries
        boundaries = h3.cell_to_boundary(row['h3_index'])

        # Create and add polygon for each hexagon
        folium.Polygon(
            locations=boundaries,
            color=get_color(row['count']),
            fill=True,
            fill_color=get_color(row['count']),
            fill_opacity=0.9,
            popup=f"Count: {row['count']}",
            weight=1
        ).add_to(m)

    # Create HTML for bilingual title
    title_html = f'''
    <h3 align="center" style="font-size:20px">
        <b>{japanese_title}</b><br>
        <span style="font-size:16px">{english_title}</span>
    </h3>
    '''

    # Add title to map
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [6]:
def main(api_key):
    # Get sakura locations
    print("Fetching sakura locations...")
    df = get_sakura_locations(api_key)

    # Create density map
    print("Creating density map...")
    hex_df = create_density_map(df)

    # Calculate center point (Tokyo Station)
    center_lat, center_lng = 35.6812, 139.7671

    # Create visualization
    print("Generating map visualization...")
    m = visualize_density_with_title(hex_df, center_lat, center_lng)

    # Save map
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f'tokyo_sakura_density_{timestamp}.html'
    m.save(output_file)
    print(f"Map saved as {output_file}")

    return df, hex_df, m

In [7]:
# Usage example:
if __name__ == "__main__":
    api_key = "YOUR_API_KEY_HERE"
    df, hex_df, m = main(api_key)

Fetching sakura locations...
Creating density map...
Generating map visualization...
Map saved as tokyo_sakura_density_20241105_011323.html


In [8]:
m

### 5. Understanding the Visualization 📊

#### Color Scale Design
* Light pink: Low density (1-5 trees)
* Medium pink: Moderate density (6-15 trees)
* Deep pink: High density (16-30 trees)
* Fuchsia: Maximum concentration (>30 trees)

#### Notable Locations
* 上野公園 (Ueno Park): Highest concentration
* 新宿御苑 (Shinjuku Gyoen): Medium-high density
* 隅田公園 (Sumida Park): Significant clusters
* 目黒川 (Meguro River): Linear distribution

### 6. Temporal Analysis 📅

#### Blooming Patterns
* Start: Late March (開花 - Kaika)
* Peak: First week of April (満開 - Mankai)
* Duration: 1-2 weeks
* Location variations based on microclimate

### 7. Spatial Insights 🔍

#### Distribution Patterns
* Concentration in public parks (公園)
* Correlation with historical green spaces
* Urban heat island effects
* Microclimate influences

#### Practical Applications
* 花見 (Hanami) planning
* Tourism management
* Tree conservation
* Climate impact studies

### 8. H3 Best Practices 🎯

#### Resolution Selection Guide
* Res 8-9: Urban analysis (recommended for our use case)
* Res 6-7: Regional patterns
* Res 10+: Micro-analysis
* Consider data density and visualization goals

#### Performance Optimization
* Index caching
* Efficient aggregation
* Edge handling
* Set compaction

### 9. Additional Resources 📚
* [H3 Documentation](https://h3geo.org/)
* [Uber Engineering Blog](https://eng.uber.com/h3/)
* [Tokyo Tourism Data](https://www.gotokyo.org/)
* [H3 Visualization Guide](https://github.com/uber/h3)

### さくらの季節 (Sakura Season) 🌸
桜の花の下で (Under the cherry blossoms)
一期一会 (One life, one encounter)
今この瞬間を大切に (Cherishing this moment)

See you tomorrow for another mapping adventure! 🗺️

In [9]:
#@title Follow Me!
%%html

<div style="background: linear-gradient(90deg, #1a1a1a, #2d2d2d); color: white; padding: 2rem; border-radius: 10px; font-family: 'Roboto', sans-serif; margin-top: 2rem;">
    <h2 style="text-align: center; margin-bottom: 1rem;">
        <span style="display: block; color: white; font-size: 2.7rem; margin-bottom: 0.5rem;">ありがとうございます！</span>
        <span style="display: block; color: white; font-size: 1.8rem;">🌍 Thank you for making it this far! 🎌</span>
    </h2>

    <p style="font-size: 1.2rem; margin-bottom: 1.5rem;">If you're interested in the world of geoinformatics and want to keep learning, connect with me! | 一緒に学びましょう！</p>

    <div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-bottom: 1rem;">
        <a href="https://x.com/GastonEchenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #1DA1F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M18.244 2.25h3.308l-7.227 8.26 8.502 11.24H16.17l-5.214-6.817L4.99 21.75H1.68l7.73-8.835L1.254 2.25H8.08l4.713 6.231zm-1.161 17.52h1.833L7.084 4.126H5.117z"/>
            </svg>
            <span>@GastonEchenique</span>
        </a>

        <a href="https://www.linkedin.com/in/gaston-echenique/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #0A66C2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.447 20.452h-3.554v-5.569c0-1.328-.027-3.037-1.852-3.037-1.853 0-2.136 1.445-2.136 2.939v5.667H9.351V9h3.414v1.561h.046c.477-.9 1.637-1.85 3.37-1.85 3.601 0 4.267 2.37 4.267 5.455v6.286zM5.337 7.433c-1.144 0-2.063-.926-2.063-2.065 0-1.138.92-2.063 2.063-2.063 1.14 0 2.064.925 2.064 2.063 0 1.139-.925 2.065-2.064 2.065zm1.782 13.019H3.555V9h3.564v11.452zM22.225 0H1.771C.792 0 0 .774 0 1.729v20.542C0 23.227.792 24 1.771 24h20.451C23.2 24 24 23.227 24 22.271V1.729C24 .774 23.2 0 22.222 0h.003z"/>
            </svg>
            <span>Gastón Echenique</span>
        </a>

        <a href="https://github.com/oechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #333; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.626 0-12 5.373-12 12 0 5.302 3.438 9.8 8.207 11.387.599.111.793-.261.793-.577v-2.234c-3.338.726-4.033-1.416-4.033-1.416-.546-1.387-1.333-1.756-1.333-1.756-1.089-.745.083-.729.083-.729 1.205.084 1.839 1.237 1.839 1.237 1.07 1.834 2.807 1.304 3.492.997.107-.775.418-1.305.762-1.604-2.665-.305-5.467-1.334-5.467-5.931 0-1.311.469-2.381 1.236-3.221-.124-.303-.535-1.524.117-3.176 0 0 1.008-.322 3.301 1.23.957-.266 1.983-.399 3.003-.404 1.02.005 2.047.138 3.006.404 2.291-1.552 3.297-1.23 3.297-1.23.653 1.653.242 2.874.118 3.176.77.84 1.235 1.911 1.235 3.221 0 4.609-2.807 5.624-5.479 5.921.43.372.823 1.102.823 2.222v3.293c0 .319.192.694.801.576 4.765-1.589 8.199-6.086 8.199-11.386 0-6.627-5.373-12-12-12z"/>
            </svg>
            <span>oechenique</span>
        </a>

        <a href="https://oechenique.github.io/geoanalytics/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #2ecc71; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.627 0-12 5.373-12 12s5.373 12 12 12 12-5.373 12-12-5.373-12-12-12zm1 16.057v-3.057h2.994c-.059 1.143-.212 2.24-.456 3.279-.823-.12-1.674-.188-2.538-.222zm1.957 2.162c-.499 1.33-1.159 2.497-1.957 3.456v-3.62c.666.028 1.319.081 1.957.164zm-1.957-7.219v-3.015c.868-.034 1.721-.103 2.548-.224.238 1.027.389 2.111.446 3.239h-2.994zm0-5.014v-3.661c.806.969 1.471 2.15 1.971 3.496-.642.084-1.3.137-1.971.165zm2.703-3.267c1.237.496 2.354 1.228 3.29 2.146-.642.234-1.311.442-2.019.607-.344-.992-.775-1.91-1.271-2.753zm-7.241 13.56c-.244-1.039-.398-2.136-.456-3.279h2.994v3.057c-.865.034-1.714.102-2.538.222zm2.538 1.776v3.62c-.798-.959-1.458-2.126-1.957-3.456.638-.083 1.291-.136 1.957-.164zm-2.994-7.055c.057-1.128.207-2.212.446-3.239.827.121 1.68.19 2.548.224v3.015h-2.994zm1.024-5.179c.5-1.346 1.165-2.527 1.97-3.496v3.661c-.671-.028-1.329-.081-1.97-.165zm-2.005-.35c-.708-.165-1.377-.373-2.018-.607.937-.918 2.053-1.65 3.29-2.146-.496.844-.927 1.762-1.272 2.753zm-.549 1.918c-.264 1.151-.434 2.36-.492 3.611h-3.933c.165-1.658.739-3.197 1.617-4.518.88.361 1.816.67 2.808.907zm.009 9.262c-.988.236-1.92.542-2.797.9-.89-1.328-1.471-2.879-1.637-4.551h3.934c.058 1.265.231 2.488.5 3.651zm.553 1.917c.342.976.768 1.881 1.257 2.712-1.223-.49-2.326-1.211-3.256-2.115.636-.229 1.299-.435 1.999-.597zm9.924 0c.7.163 1.362.367 1.999.597-.931.903-2.034 1.625-3.257 2.116.489-.832.915-1.737 1.258-2.713zm.553-1.917c.27-1.163.442-2.386.501-3.651h3.934c-.167 1.672-.748 3.223-1.638 4.551-.877-.358-1.81-.664-2.797-.9zm.501-5.651c-.058-1.251-.229-2.46-.492-3.611.992-.237 1.929-.546 2.809-.907.877 1.321 1.451 2.86 1.616 4.518h-3.933z"/>
            </svg>
            <span>GeoAnalytics</span>
        </a>

        <a href="https://discord.com/users/gastonechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #5865F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.317 4.37a19.791 19.791 0 0 0-4.885-1.515a.074.074 0 0 0-.079.037c-.21.375-.444.864-.608 1.25a18.27 18.27 0 0 0-5.487 0a12.64 12.64 0 0 0-.617-1.25a.077.077 0 0 0-.079-.037A19.736 19.736 0 0 0 3.677 4.37a.07.07 0 0 0-.032.027C.533 9.046-.32 13.58.099 18.057a.082.082 0 0 0 .031.057a19.9 19.9 0 0 0 5.993 3.03a.078.078 0 0 0 .084-.028a14.09 14.09 0 0 0 1.226-1.994a.076.076 0 0 0-.041-.106a13.107 13.107 0 0 1-1.872-.892a.077.077 0 0 1-.008-.128a10.2 10.2 0 0 0 .372-.292a.074.074 0 0 1 .077-.01c3.928 1.793 8.18 1.793 12.062 0a.074.074 0 0 1 .078.01c.12.098.246.198.373.292a.077.077 0 0 1-.006.127a12.299 12.299 0 0 1-1.873.892a.077.077 0 0 0-.041.107c.36.698.772 1.362 1.225 1.993a.076.076 0 0 0 .084.028a19.839 19.839 0 0 0 6.002-3.03a.077.077 0 0 0 .032-.054c.5-5.177-.838-9.674-3.549-13.66a.061.061 0 0 0-.031-.03zM8.02 15.33c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.956-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.956 2.418-2.157 2.418zm7.975 0c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.955-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.946 2.418-2.157 2.418z"/>
            </svg>
            <span>Gaston | ガストン</span>
        </a>
    </div>

    <p style="font-size: 1rem; color: #888;">💡 頑張りましょう！Don't miss more content about GIS, Data Science and Geoinformatics!</p>

    <div style="margin-top: 1rem; padding-top: 1rem; border-top: 1px solid #444; font-size: 0.9rem; color: #666;">
        Made with 🗺️ & 💻 by Gaston Echenique | エチェニケ・ガストン
    </div>
</div>